In [1]:
import pandas as pd

df = pd.read_csv("../data/loans.csv")
df.head()


,Loan ID,Status,Client ID,Debtor Identifier,Debtor Identifier Type,Debtor Country,Trust ID,Amount,Created At,Accepted At,Refused At,Repaid At,Due Date,Insurance Status
0,ccba7b5961ac84c6bc09ba20b0497bd666ac10a7ecf123...,REPAID,2477304f4867e7ea86fd4414e0f845c0e4bd06516fe687...,17c277f8d264ccec868dc55add915dc93fd4ae4bd779cf...,siren,FR,e98a46aea01b6f55341744cdddbba3d6a88ab1e7d366eb...,13643.52,2025-01-24T09:52:41.912,2025-01-24T09:52:54.08,NaN,2025-03-06T05:05:38.362,2025-03-25T08:00:00,SETTLED
1,ea43f967166a7809d0e0f27fff33a3732fa600513d1b5f...,REPAID,a57af78c8caa2a9c6efbf8d9535f34441319c66ba56803...,494f60d81e3f8e6effbf093a1c2c6d0a99ca9d561bf624...,siren,FR,e98a46aea01b6f55341744cdddbba3d6a88ab1e7d366eb...,58.97,2025-08-27T07:51:48.154,2025-08-27T09:59:54.827,NaN,2025-09-30T10:30:19,01/10/2025,NaN
2,1d39f1f6c61cef090d31c552b0d3e511280ba64c2fb69a...,REPAID,dd3a54bd37885757d4c4b3dbd85c5fdcffa351fe2d1680...,8ab1efec5e005f628c0e5793276ddaf1e3693cf880a0a9...,cif,ES,e98a46aea01b6f55341744cdddbba3d6a88ab1e7d366eb...,6654.03,2025-02-24T09:13:01.171,2025-02-24T16:51:01.077,NaN,2025-06-03T18:32:50,2025-05-20T09:12:33.344,REFUSED
3,f03b65936792e9d35e66db0572aa43cdc5f2d33d75fcd0...,REPAID,dd3a54bd37885757d4c4b3dbd85c5fdcffa351fe2d1680...,8563f72a004fcba1d5ae23410ccf82a6d8bcc85fb6ccfd...,kvk,NL,c4ed1a68f3ad7b3c85c4400e688e4dd3dcfe4da53d171d...,4452.80,2025-10-07T06:19:38.206,2025-10-07T06:19:47.087,NaN,2025-10-29T15:34:56,2025-11-01T06:19:09.316,SETTLED
4,70290cf7ced390115c4443cab5f519fcf8ac52a11dafbd...,REPAID,dd3a54bd37885757d4c4b3dbd85c5fdcffa351fe2d1680...,17c277f8d264ccec868dc55add915dc93fd4ae4bd779cf...,siret,FR,e98a46aea01b6f55341744cdddbba3d6a88ab1e7d366eb...,966.08,2025-07-29T12:21:45.349,2025-07-29T12:21:54.306,NaN,2025-09-05T14:43:14,2025-09-22T12:21:41.397,SETTLED


In [2]:
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
)

df.columns.tolist()

['loan_id',
 'status',
 'client_id',
 'debtor_identifier',
 'debtor_identifier_type',
 'debtor_country',
 'trust_id',
 'amount',
 'created_at',
 'accepted_at',
 'refused_at',
 'repaid_at',
 'due_date',
 'insurance_status']

In [3]:
import numpy as np
import pandas as pd

OWNER_COL = "client_id"
AMOUNT_COL = "amount"

df[AMOUNT_COL] = pd.to_numeric(df[AMOUNT_COL], errors="coerce")

base = df.loc[df[AMOUNT_COL].notna(), [OWNER_COL, AMOUNT_COL]].copy()

summary = (
    base.groupby(OWNER_COL)[AMOUNT_COL]
    .agg(
        loan_count="count",
        total_loan_amount="sum",
        min_loan_amount="min",
        p25_loan_amount=lambda s: s.quantile(0.25),
        median_loan_amount=lambda s: s.quantile(0.50),
        p75_loan_amount=lambda s: s.quantile(0.75),
        avg_loan_amount="mean",
        max_loan_amount="max",
        stddev_loan_amount="std",
    )
    .reset_index()
    .rename(columns={OWNER_COL: "owner_id"})
)

grand_total = summary["total_loan_amount"].sum()
summary["pct_of_total_amount"] = np.where(
    grand_total == 0, np.nan, summary["total_loan_amount"] / grand_total
)

summary = summary.sort_values("total_loan_amount", ascending=False).reset_index(drop=True)

summary.head()

,owner_id,loan_count,total_loan_amount,min_loan_amount,p25_loan_amount,median_loan_amount,p75_loan_amount,avg_loan_amount,max_loan_amount,stddev_loan_amount,pct_of_total_amount
0,dd3a54bd37885757d4c4b3dbd85c5fdcffa351fe2d1680...,27218,2.345904e+08,7.69,2380.0000,7490.000,13307.9975,8618.942294,203570.80,7346.602090,0.808498
1,82406cd2b0a1254eac2268f9877c686b8f57d1c16b8fa6...,5323,2.530327e+07,0.58,1442.2800,3934.080,6460.8550,4753.572688,107980.55,5479.722213,0.087206
2,ace3952a4027ddd5b5572a3e5cb1c16afb6b224e06c178...,3072,2.445055e+07,0.05,817.6975,2398.900,7055.5600,7959.163776,470512.85,21603.377555,0.084267
3,2477304f4867e7ea86fd4414e0f845c0e4bd06516fe687...,338,3.982034e+06,215.46,8597.0250,12789.025,15183.6000,11781.164941,27000.00,5201.817164,0.013724
4,a57af78c8caa2a9c6efbf8d9535f34441319c66ba56803...,32132,1.829395e+06,0.02,32.9700,53.405,70.9400,56.933730,5589.96,65.664348,0.006305


In [4]:
summary_display = summary.copy()

summary_display = summary_display.rename(columns={
    "owner_id": "Owner ID",
    "total_loan_amount": "Total Loan Amount",
    "loan_count": "Loan Count",
    "median_loan_amount": "Median Loan Amount",
    "min_loan_amount": "Min Loan Amount",
    "avg_loan_amount": "Avg Loan Amount",
    "max_loan_amount": "Max Loan Amount",
    "stddev_loan_amount": "Standard Deviation Amount",
    "pct_of_total_amount": "% of Total Amount",
    "p25_loan_amount": "P25 Loan Amount",
    "p75_loan_amount": "P75 Loan Amount",
})

summary_display["Total Loan Amount"] = summary_display["Total Loan Amount"].map(
    lambda x: f"{x:,.0f}" if pd.notna(x) else ""
)

summary_display["Loan Count"] = summary_display["Loan Count"].map(
    lambda x: f"{int(x):,}" if pd.notna(x) else ""
)

summary_display["Median Loan Amount"] = summary_display["Median Loan Amount"].map(
    lambda x: f"{x:,.0f}" if pd.notna(x) else ""
)

summary_display["Min Loan Amount"] = summary_display["Min Loan Amount"].map(
    lambda x: f"{x:,.2f}" if pd.notna(x) else ""
)

summary_display["Avg Loan Amount"] = summary_display["Avg Loan Amount"].map(
    lambda x: f"{x:,.0f}" if pd.notna(x) else ""
)

summary_display["Max Loan Amount"] = summary_display["Max Loan Amount"].map(
    lambda x: f"{x:,.0f}" if pd.notna(x) else ""
)

summary_display["Standard Deviation Amount"] = summary_display["Standard Deviation Amount"].map(
    lambda x: f"{x:,.0f}" if pd.notna(x) else ""
)

summary_display["% of Total Amount"] = summary_display["% of Total Amount"].map(
    lambda x: f"{x*100:.1f}%" if pd.notna(x) else ""
)

summary_display["P25 Loan Amount"] = summary_display["P25 Loan Amount"].map(
    lambda x: f"{x:,.0f}" if pd.notna(x) else ""
)

summary_display["P75 Loan Amount"] = summary_display["P75 Loan Amount"].map(
    lambda x: f"{x:,.0f}" if pd.notna(x) else ""
)

summary_display = summary_display[
    [
        "Owner ID",
        "Total Loan Amount",
        "Loan Count",
        "Median Loan Amount",
        "Min Loan Amount",
        "Avg Loan Amount",
        "Max Loan Amount",
        "Standard Deviation Amount",
        "% of Total Amount",
        "P25 Loan Amount",
        "P75 Loan Amount",
    ]
]

summary_display

,Owner ID,Total Loan Amount,Loan Count,Median Loan Amount,Min Loan Amount,Avg Loan Amount,Max Loan Amount,Standard Deviation Amount,% of Total Amount,P25 Loan Amount,P75 Loan Amount
0,dd3a54bd37885757d4c4b3dbd85c5fdcffa351fe2d1680...,"234,590,371","27,218","7,490",7.69,"8,619","203,571","7,347",80.8%,"2,380","13,308"
1,82406cd2b0a1254eac2268f9877c686b8f57d1c16b8fa6...,"25,303,267","5,323","3,934",0.58,"4,754","107,981","5,480",8.7%,"1,442","6,461"
2,ace3952a4027ddd5b5572a3e5cb1c16afb6b224e06c178...,"24,450,551","3,072","2,399",0.05,"7,959","470,513","21,603",8.4%,818,"7,056"
3,2477304f4867e7ea86fd4414e0f845c0e4bd06516fe687...,"3,982,034",338,"12,789",215.46,"11,781","27,000","5,202",1.4%,"8,597","15,184"
4,a57af78c8caa2a9c6efbf8d9535f34441319c66ba56803...,"1,829,395","32,132",53,0.02,57,"5,590",66,0.6%,33,71


In [5]:
import pandas as pd
import numpy as np
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output

OWNER_COL = "client_id"
AMOUNT_COL = "amount"
DATE_COL  = "created_at" 

df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce")
df[AMOUNT_COL] = pd.to_numeric(df[AMOUNT_COL], errors="coerce")

base = df.loc[
    df[DATE_COL].notna() & df[AMOUNT_COL].notna(),
    [OWNER_COL, AMOUNT_COL, DATE_COL]
].copy()

date_min = base[DATE_COL].min()
date_max = base[DATE_COL].max()

if pd.isna(date_min) or pd.isna(date_max):
    raise ValueError(f"No valid dates found in '{DATE_COL}'. Try another date column.")

from_date = widgets.DatePicker(
    description="From:",
    value=date_min.date()
)

to_date = widgets.DatePicker(
    description="To:",
    value=date_max.date()
)

chart_selector = widgets.Dropdown(
    options=[
        "Table: Loan Distribution by Owner",
        "Bar Chart: Total Loan Amount by Owner",
        "Bar Chart: Number of Loans by Owner",
        "Histogram: Loan Amount Distribution"
    ],
    value="Table: Loan Distribution by Owner",
    description="View:",
    layout=widgets.Layout(width="520px")
)

output = widgets.Output()

def build_summary_with_distribution(data):
    s = (
        data.groupby(OWNER_COL)[AMOUNT_COL]
        .agg(
            loan_count="count",
            total_loan_amount="sum",
            min_loan_amount="min",
            p25_loan_amount=lambda x: x.quantile(0.25),
            median_loan_amount=lambda x: x.quantile(0.50),
            p75_loan_amount=lambda x: x.quantile(0.75),
            avg_loan_amount="mean",
            max_loan_amount="max",
            stddev_loan_amount="std",
        )
        .reset_index()
        .rename(columns={OWNER_COL: "Owner ID"})
        .sort_values("total_loan_amount", ascending=False)
        .reset_index(drop=True)
    )

    total = s["total_loan_amount"].sum()
    s["pct_of_total_amount"] = np.where(total == 0, np.nan, s["total_loan_amount"] / total)
    return s

def format_table(s):

    out_tbl = s.rename(columns={
        "total_loan_amount": "Total Loan Amount",
        "loan_count": "Loan Count",
        "median_loan_amount": "Median Loan Amount",
        "min_loan_amount": "Min Loan Amount",
        "avg_loan_amount": "Avg Loan Amount",
        "max_loan_amount": "Max Loan Amount",
        "stddev_loan_amount": "Standard Deviation Amount",
        "pct_of_total_amount": "% of Total Amount",
        "p25_loan_amount": "P25 Loan Amount",
        "p75_loan_amount": "P75 Loan Amount",
    }).copy()

    out_tbl["Total Loan Amount"] = out_tbl["Total Loan Amount"].map(lambda x: f"{x:,.0f}" if pd.notna(x) else "")
    out_tbl["Loan Count"] = out_tbl["Loan Count"].map(lambda x: f"{int(x):,}" if pd.notna(x) else "")
    out_tbl["Median Loan Amount"] = out_tbl["Median Loan Amount"].map(lambda x: f"{x:,.0f}" if pd.notna(x) else "")
    out_tbl["Min Loan Amount"] = out_tbl["Min Loan Amount"].map(lambda x: f"{x:,.2f}" if pd.notna(x) else "")
    out_tbl["Avg Loan Amount"] = out_tbl["Avg Loan Amount"].map(lambda x: f"{x:,.0f}" if pd.notna(x) else "")
    out_tbl["Max Loan Amount"] = out_tbl["Max Loan Amount"].map(lambda x: f"{x:,.0f}" if pd.notna(x) else "")
    out_tbl["Standard Deviation Amount"] = out_tbl["Standard Deviation Amount"].map(lambda x: f"{x:,.0f}" if pd.notna(x) else "")
    out_tbl["% of Total Amount"] = out_tbl["% of Total Amount"].map(lambda x: f"{x*100:.1f}%" if pd.notna(x) else "")
    out_tbl["P25 Loan Amount"] = out_tbl["P25 Loan Amount"].map(lambda x: f"{x:,.0f}" if pd.notna(x) else "")
    out_tbl["P75 Loan Amount"] = out_tbl["P75 Loan Amount"].map(lambda x: f"{x:,.0f}" if pd.notna(x) else "")

    out_tbl = out_tbl[
        [
            "Owner ID",
            "Total Loan Amount",
            "Loan Count",
            "Median Loan Amount",
            "Min Loan Amount",
            "Avg Loan Amount",
            "Max Loan Amount",
            "Standard Deviation Amount",
            "% of Total Amount",
            "P25 Loan Amount",
            "P75 Loan Amount",
        ]
    ]
    return out_tbl

def render(_=None):
    with output:
        clear_output(wait=True)

        if from_date.value is None or to_date.value is None:
            print("Please select both From and To dates.")
            return

        start = pd.Timestamp(from_date.value)
        end = pd.Timestamp(to_date.value)

        if start > end:
            print("From date must be earlier than To date.")
            return

        filtered = base.loc[(base[DATE_COL] >= start) & (base[DATE_COL] <= end)].copy()

        if filtered.empty:
            print("No data in the selected timeframe. Try widening the date range.")
            return

        summary = build_summary_with_distribution(filtered)

        if chart_selector.value == "Table: Loan Distribution by Owner":
            display(format_table(summary))

        elif chart_selector.value == "Bar Chart: Total Loan Amount by Owner":
            fig = px.bar(
                summary,
                x="Owner ID",
                y="total_loan_amount",
                title=f"Total Loan Amount by Owner ({DATE_COL} timeframe)",
            )
            fig.update_layout(yaxis_tickformat=",", xaxis_title="Owner ID", yaxis_title="Total Loan Amount")
            fig.show()

        elif chart_selector.value == "Bar Chart: Number of Loans by Owner":
            fig = px.bar(
                summary,
                x="Owner ID",
                y="loan_count",
                title=f"Loan Count by Owner ({DATE_COL} timeframe)",
            )
            fig.update_layout(xaxis_title="Owner ID", yaxis_title="Loan Count")
            fig.show()

        elif chart_selector.value == "Histogram: Loan Amount Distribution":
            fig = px.histogram(
                filtered,
                x=AMOUNT_COL,
                nbins=50,
                title=f"Loan Amount Distribution ({DATE_COL} timeframe)",
            )
            fig.update_layout(xaxis_title="Loan Amount", yaxis_title="Number of Loans", xaxis_tickformat=",")
            fig.show()


from_date.observe(render, names="value")
to_date.observe(render, names="value")
chart_selector.observe(render, names="value")

display(widgets.HBox([from_date, to_date]), chart_selector, output)
render()


Dropdown(description='View:', layout=Layout(width='520px'), options=('Table: Loan Distribution by Owner', 'Bar…

Output()